In [1]:
import sys
from functools import total_ordering

class Vertex():
    """
    ---- simplest implementation
    self.id
    self.adjacent = dict {adjacent vertex: edge}
    
    --- methods
    self.add_neighbor --- neighbor + edge(weight) 
    __str__ --- self.id + neighbors
    
    """
    def __init__(self, node):
        self.id = node
        self.adjacent = {}
        self.distance = sys.maxsize
        self.visited = False
        self.previous = None
    
    def add_neighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] = weight
        
    def get_edge_value(self, neighbor):
        return self.adjacent[neighbor]
        
    def __str__(self):
        neighbor = []
        for vertex in self.adjacent:
            neighbor.append(vertex.id)
        neighbor = " ".join(neighbor)
        
        return ("id is {} neighbors are {}".format(self.id, neighbor))
    
    def __lt__(self, other):  # we need to this to compare vertex and vertex
        return self.distance < other.distance
    
class Graph():
    """
    --- simplest implementation
    self.vert_dict = dict {self.id : vertex}
    
    --- methods
    add_vertex ---- supply just id
    add_edge ---- two ids and weight for edge
    
    """
    
    def __init__(self):
        self.vert_dict = {}
        
    def __iter__(self): #### Make a Graph iterative
        return iter(self.vert_dict.values())
        
    def add_vertex(self, node):
        self.vert_dict[node] = Vertex(node)
        
    def add_edge(self, frm, to, cost = 0):
        # check if there is frm and to in self.vert_dict
        if frm not in self.vert_dict:
            self.vert_dict[frm] = Vertex(frm)
            
        if to not in self.vert_dict:
            self.vert_dict[to] = Vertex(to)
            
        # add edge
        self.vert_dict[frm].add_neighbor(self.vert_dict[to], cost)
        self.vert_dict[to].add_neighbor(self.vert_dict[frm], cost)

In [2]:
g = Graph()

g.add_vertex('a')
g.add_vertex('b')
g.add_vertex('c')
g.add_vertex('d')
g.add_vertex('e')
g.add_vertex('f')

g.add_edge('a', 'b', 7)  
g.add_edge('a', 'c', 9)
g.add_edge('a', 'f', 14)
g.add_edge('b', 'c', 10)
g.add_edge('b', 'd', 15)
g.add_edge('c', 'd', 11)
g.add_edge('c', 'f', 2)
g.add_edge('d', 'e', 6)
g.add_edge('e', 'f', 9)

In [3]:
import heapq

def dijkstra(aGraph, start):
    """
    - initialize, set-up staring node (vertex), build a priority queue
    - while loop
        - get a vertex with the smallest distance
        - update distance
        - rebuild heap => back
    
    """
    print ("Dijkstra's shortest path")
    
    # init
    start.distance = 0
    
    # create a heap
    unvisited_queue = [(v.distance, v) for v in aGraph]
    heapq.heapify(unvisited_queue)
    
    while len(unvisited_queue) > 0:
        # pop a vertex with the smallest distance
        uv = heapq.heappop(unvisited_queue)
        print ("------------------------------------------------")
        current = uv[1] # vertex
        current.visited = True
        
        # visit neighbors
        for n in current.adjacent: # n is vertex
            if n.visited:
                continue
                
            # update distance
            new_dist = current.distance + current.get_edge_value(n)
            
            if new_dist < n.distance:
                n.distance = new_dist
                n.previous = current
                print ("updated: current = {} next = {} new_dist = {}".format(
                current.id, n.id, n.distance))
        
        while len(unvisited_queue) > 0:
            heapq.heappop(unvisited_queue)

        unvisited_queue = [(v.distance, v) for v in aGraph if not v.visited]
        heapq.heapify(unvisited_queue)

In [4]:
def shortest(v, path):
    ''' make shortest path from v.previous'''
    if v.previous:
        path.append(v.previous.id)
        shortest(v.previous, path)
    return

In [5]:
dijkstra(g, g.vert_dict['a']) 

Dijkstra's shortest path
------------------------------------------------
updated: current = a next = b new_dist = 7
updated: current = a next = c new_dist = 9
updated: current = a next = f new_dist = 14
------------------------------------------------
updated: current = b next = d new_dist = 22
------------------------------------------------
updated: current = c next = d new_dist = 20
updated: current = c next = f new_dist = 11
------------------------------------------------
updated: current = f next = e new_dist = 20
------------------------------------------------
------------------------------------------------


In [6]:
for i in g:
    print (i)

id is a neighbors are b c f
id is b neighbors are a c d
id is c neighbors are a b d f
id is d neighbors are b c e
id is e neighbors are d f
id is f neighbors are a c e


In [7]:
g.vert_dict

{'a': <__main__.Vertex at 0x2a0c8ae5ac8>,
 'b': <__main__.Vertex at 0x2a0c8ae5ba8>,
 'c': <__main__.Vertex at 0x2a0c8ae5c18>,
 'd': <__main__.Vertex at 0x2a0c8ae5c50>,
 'e': <__main__.Vertex at 0x2a0c8ae5c88>,
 'f': <__main__.Vertex at 0x2a0c8ae5cc0>}

In [8]:
for t in ['d','e','f']:
    target = g.vert_dict[t]
    path = [t]
    shortest(target, path)
    print ('The shortest path for %s : %s' %(t, path[::-1]))

The shortest path for d : ['a', 'c', 'd']
The shortest path for e : ['a', 'c', 'f', 'e']
The shortest path for f : ['a', 'c', 'f']
